In [1]:
import numpy as np
import pandas as pd
import gc # Use gc.collect() to release memory usage
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read publicly available data (>4mil entries)
data = pd.read_csv('GKX_20201231.csv')

In [3]:
start_date, end_date = 19570131, 20161231

In [4]:
# This is all the data that will be used for recursive training
data = data[(data['DATE'] >= start_date) & (data['DATE'] <= end_date)].reset_index(drop=True)

# Change date format for grouping later; offsets.MonthEnd(0) means dates need not be adjusted. They were adjusted in the dataset
data['DATE'] = pd.to_datetime(data['DATE'], format='%Y%m%d') + pd.offsets.MonthEnd(0)

# In case "data" is needed later
# data_copy = data.copy()

In [5]:
characteristics = list(set(data.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))
characteristics.sort()

In [6]:
# Fill NA entries with cross-sectional median
for ch in characteristics:
    data[ch] = data.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))

# Fill the rest with 0
for ch in characteristics:
    data[ch] = data[ch].fillna(0)

print(data.columns[data.isnull().sum() != 0])

Index(['prc', 'mve0', 'sic2'], dtype='object')


In [7]:
# Load macroeconomic predictors data. There are eight of them
data_ma = pd.read_excel('PredictorData2022.xlsx')
# The format of dates in macro predictors is YYYYMM instead of YYYYMMDD
data_ma = data_ma[(data_ma['yyyymm'] >= start_date//100) & (data_ma['yyyymm'] <= end_date//100)].reset_index(drop=True)

In [8]:
# Construct predictors
# Index, ntis, tbl, svar are given in the dataset. The remaining four are calculated using other columns
ma_predictors = ['dp_sp','ep_sp','bm_sp','ntis','tbl','tms','dfy','svar']

# data_ma['Index'] is already float64
# data_ma['Index'] = data_ma['Index'].str.replace(',','').astype('float64')
data_ma['dp_sp'] = data_ma['D12'] / data_ma['Index']
data_ma['ep_sp'] = data_ma['E12'] / data_ma['Index']
data_ma.rename({'b/m':'bm_sp'},axis=1,inplace=True)
data_ma['tms'] = data_ma['lty'] - data_ma['tbl']
data_ma['dfy'] = data_ma['BAA'] - data_ma['AAA']

# This removes all the intermediate columns and leaves only the eight predictors, date, and risk-free rate column
data_ma = data_ma[['yyyymm'] + ma_predictors + ['Rfree']]
data_ma['yyyymm'] = pd.to_datetime(data_ma['yyyymm'], format='%Y%m') + pd.offsets.MonthEnd(0)

In [9]:
data_ma

,yyyymm,dp_sp,ep_sp,bm_sp,ntis,tbl,tms,dfy,svar,Rfree
0,1957-01-31,0.038834,0.076178,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,0.0027
1,1957-02-28,0.040068,0.078672,0.584994,0.030173,0.0310,0.0018,0.0080,0.001056,0.0024
2,1957-03-31,0.039220,0.077080,0.599819,0.026600,0.0308,0.0023,0.0077,0.000330,0.0023
3,1957-04-30,0.037822,0.074479,0.576098,0.027421,0.0307,0.0038,0.0077,0.000302,0.0025
4,1957-05-31,0.036475,0.071966,0.564039,0.028849,0.0306,0.0042,0.0078,0.000482,0.0026
...,...,...,...,...,...,...,...,...,...,...
715,2016-08-31,0.020653,0.040704,0.315197,-0.030782,0.0030,0.0156,0.0092,0.000279,0.0002
716,2016-09-30,0.020766,0.041088,0.316794,-0.032603,0.0029,0.0167,0.0090,0.001673,0.0002
717,2016-10-31,0.021283,0.042758,0.319688,-0.029034,0.0033,0.0187,0.0087,0.000364,0.0002
718,2016-11-30,0.020682,0.042173,0.303286,-0.027452,0.0045,0.0222,0.0085,0.000946,0.0001


In [10]:
# Get dummies for SIC code
def get_sic_dummies(data):
    sic_dummies = pd.get_dummies(data['sic2'].fillna(999).astype(int), prefix='sic').drop('sic_999', axis=1)
    data = pd.concat([data, sic_dummies], axis=1)
    data.drop(['prc', 'SHROUT', 'mve0', 'sic2'], inplace=True, axis=1)
    return data

In [11]:
data = get_sic_dummies(data)
print(data.shape)

(3762139, 171)


In [12]:
# training 1957 - 1974
# validation 1975 - 1986
# test 1987 - 2016
start_val = np.datetime64('1975-01-31')
start_test = np.datetime64('1987-01-31')
end_test = np.datetime64('1987-12-31')

In [14]:
def interactions(data, data_ma, characteristics, ma_predictors):

    data_ma_long = pd.merge(data[['DATE']], data_ma, left_on='DATE', right_on='yyyymm', how='left').reset_index(drop=True)
    
    # Important! Reset index so that we can construct the interaction columns correctly
    data = data.reset_index(drop=True)
    
    # Change RET to excess return
    data.loc[:, 'RET'] = data.loc[:, 'RET'] - data_ma_long.loc[:, 'Rfree']
    
    # Data PRE-processing: cross-sectional rank transformation
    # data has all columns (including DATE) and all rows
    for fc in characteristics:
        data[fc] = data.groupby('DATE')[fc].rank()
        data[fc] = data.groupby('DATE')[fc].transform(lambda x: ((2 * (x - x.min())) / (x.max() - x.min())) - 1
                                                     if x.max() - x.min() != 0 else 0)
    
    # Construct interactions
    interactions = []
    for fc in characteristics:
        for mp in ma_predictors:
            data[fc + '*' + mp] = data.loc[:, fc] * data_ma_long.loc[:, mp]
            interactions.append(fc + '*' + mp)
    
    # Also normalize the interactions so there are no super small or large numbers
    for item in interactions:
        data[item] = data.groupby('DATE')[item].transform(lambda x: ((2 * (x - x.min())) / (x.max() - x.min())) - 1
                                                     if x.max() - x.min() != 0 else 0)
        
    # 94 (chars) * 8 (macro) + 94 (chars) + 74 (industry) = 920
    features = list(set(data.columns).difference({'permno', 'DATE', 'RET'}))
    
    # Get x and y
    x = data[features]
    y = pd.DataFrame(data['RET'], columns=['RET'])
    
    # Get top 1k
    x_t_sorted = data.sort_values(by='mvel1', ascending=False).groupby('DATE').head(1000).reset_index(drop=True)
    x_t = x_t_sorted[features]
    y_t = pd.DataFrame(x_t_sorted['RET'], columns=['RET'])
    
    # Get bot 1k; without ascending=False, ordered in ascending order
    x_b_sorted = data.sort_values(by='mvel1').groupby('DATE').head(1000).reset_index(drop=True)
    x_b = x_b_sorted[features]
    y_b = pd.DataFrame(x_b_sorted['RET'], columns=['RET'])
    
    print(x.shape, y.shape, x_t.shape, y_t.shape, x_b.shape, y_b.shape)
    return x, y, x_t, y_t, x_b, y_b

In [15]:
x_train, y_train, x_train_t, y_train_t, x_train_b, y_train_b = interactions(data[data['DATE'] < start_val], 
                                                                            data_ma[data_ma['yyyymm'] < start_val],
                                                                            characteristics, ma_predictors)

x_val, y_val, x_val_t, y_val_t, x_val_b, y_val_b = interactions(data[(data['DATE'] < start_test) & (data['DATE'] >= start_val)],
                                                                data_ma[(data_ma['yyyymm'] < start_test) & (data_ma['yyyymm'] >= start_val)],
                                                                characteristics, ma_predictors)

x_test, y_test, x_test_t, y_test_t, x_test_b, y_test_b = interactions(data[(data['DATE'] >= start_test) & (data['DATE'] <= end_test)],
                                                                      data_ma[(data_ma['yyyymm'] >= start_test) & (data_ma['yyyymm'] <= end_test)],
                                                                      characteristics, ma_predictors)
gc.collect()
    

(479467, 920) (479467, 1) (216000, 920) (216000, 1) (216000, 920) (216000, 1)
(773887, 920) (773887, 1) (144000, 920) (144000, 1) (144000, 920) (144000, 1)
(83323, 920) (83323, 1) (12000, 920) (12000, 1) (12000, 920) (12000, 1)


0

In [16]:
# Check whether there are invalid data points before proceeding to train
print(x_train.columns[x_train.isnull().sum() != 0])
print(x_val.columns[x_val.isnull().sum() != 0])
print(x_test.columns[x_test.isnull().sum() != 0])

Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')


In [18]:
x_train.head()

,std_turn*ep_sp,roeq,sic_40,cashpr*svar,ep*ep_sp,chtx*ep_sp,nincr*tms,beta*tms,age*tbl,hire*dfy,...,chempia*tms,sin*tbl,grltnoa,pchcapx_ia*dp_sp,aeavol*bm_sp,rd*tbl,sic_31,chatoia*dfy,mom6m,egr*tbl
0,-0.021968,0.0,0,0.0,0.0,0.0,0.0,0.197697,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,-0.378119,0.0
1,0.623687,0.0,0,0.0,0.0,0.0,0.0,-0.817658,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,-0.687140,0.0
2,0.272206,0.0,0,0.0,0.0,0.0,0.0,0.117083,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,-0.234165,0.0
3,0.680993,0.0,0,0.0,0.0,0.0,0.0,-0.155470,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.452975,0.0
4,-0.090735,0.0,0,0.0,0.0,0.0,0.0,0.412668,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.474088,0.0


In [19]:
x_val.head()

,std_turn*ep_sp,roeq,sic_40,cashpr*svar,ep*ep_sp,chtx*ep_sp,nincr*tms,beta*tms,age*tbl,hire*dfy,...,chempia*tms,sin*tbl,grltnoa,pchcapx_ia*dp_sp,aeavol*bm_sp,rd*tbl,sic_31,chatoia*dfy,mom6m,egr*tbl
0,-0.465377,0.552708,0,-0.274417,-0.622961,0.000000,-0.030813,-0.124664,0.869120,0.816868,...,0.833798,-1.0,0.643660,0.666391,0.762082,-1.0,0,0.728137,0.396464,-0.506555
1,-0.506945,-1.000000,0,-0.708858,-0.957051,-0.996706,-1.000000,0.243333,0.737403,-0.712605,...,-0.505316,-1.0,0.521923,0.870159,0.580752,-1.0,0,0.884432,-0.225059,-0.926912
2,0.000103,0.000000,0,0.003510,0.003510,0.000000,-0.030813,-0.122597,-0.012335,0.003407,...,0.003407,-1.0,0.002786,0.000000,-0.003717,-1.0,0,0.000000,-0.372273,0.003407
3,-0.663751,-0.600248,0,-0.618418,-0.329754,0.000000,-0.030813,-0.191234,0.332636,0.617838,...,-0.624652,-1.0,0.002786,-0.513564,0.770756,1.0,0,0.871201,0.041249,-0.629194
4,-0.888877,0.000000,0,0.003510,0.003510,0.000000,-0.030813,-0.582799,-0.012335,0.003407,...,0.003407,-1.0,0.002786,0.000000,-0.003717,-1.0,0,0.000000,0.305903,0.003407


In [20]:
x_test.head()

,std_turn*ep_sp,roeq,sic_40,cashpr*svar,ep*ep_sp,chtx*ep_sp,nincr*tms,beta*tms,age*tbl,hire*dfy,...,chempia*tms,sin*tbl,grltnoa,pchcapx_ia*dp_sp,aeavol*bm_sp,rd*tbl,sic_31,chatoia*dfy,mom6m,egr*tbl
0,0.799819,-0.002491,0,0.003098,-0.003098,0.0,-0.013764,0.003402,-0.063098,0.0,...,-0.00287,-1.0,0.0,0.002794,-0.000075,-1.0,0,-0.002643,-1.000000,0.0
1,0.272535,-0.002491,0,-0.547110,0.845108,0.0,-0.013764,0.003402,-1.000000,0.0,...,-0.00287,-1.0,0.0,0.002794,-0.000075,-1.0,0,-0.002643,0.801830,0.0
2,-0.364787,-0.002491,0,0.003098,-0.003098,0.0,-0.013764,0.003402,-0.063098,0.0,...,-0.00287,-1.0,0.0,0.002794,-0.000075,-1.0,0,-0.002643,0.577944,0.0
3,0.856195,-0.002491,0,0.003098,-0.003098,0.0,-0.013764,0.003402,-0.063098,0.0,...,-0.00287,-1.0,0.0,0.002794,-0.000075,-1.0,0,-0.002643,0.197186,0.0
4,-0.955984,-0.002491,0,0.003098,-0.003098,0.0,-0.013764,0.003402,-0.063098,0.0,...,-0.00287,-1.0,0.0,0.002794,-0.000075,-1.0,0,-0.002643,-0.724227,0.0


# RF

In [ ]:
# Optional Random Forest
# This takes too long to run
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(n_estimators=300, max_depth=6, max_features=100).fit(x_train, y_train)

# Apply NN and OLS_3 Once

In [21]:
# Define customized R^2 according to the paper; this is not the usual definition
# This R^2 will be larger than the usual R^2
def R_squared(y_true, y_pred):
    resid = tf.square(y_true - y_pred)
    denom = tf.square(y_true)
    return 1 - tf.divide(tf.reduce_sum(resid), tf.reduce_sum(denom))

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error#, r2_score

# OLS with preselected size, bm, and momentum covariates
features_3 = ['mvel1','bm','mom1m']
OLS_3 = LinearRegression().fit(x_train[features_3], y_train)

In [28]:
# Initialize to record all OLS_3 results
OLS_3_train_mse = []
OLS_3_val_mse = []
OLS_3_test_mse = []
OLS_3_train_R2 = []
OLS_3_val_R2 = []
OLS_3_test_R2 = []

OLS_3_test_t_mse = []
OLS_3_test_t_R2 = []

OLS_3_test_b_mse = []
OLS_3_test_b_R2 = []

In [29]:
OLS_3_train_mse.append(mean_squared_error(y_train, OLS_3.predict(x_train[features_3])))
OLS_3_val_mse.append(mean_squared_error(y_val, OLS_3.predict(x_val[features_3])))
OLS_3_test_mse.append(mean_squared_error(y_test, OLS_3.predict(x_test[features_3])))
OLS_3_train_R2.append(R_squared(y_train, OLS_3.predict(x_train[features_3])))
OLS_3_val_R2.append(R_squared(y_val, OLS_3.predict(x_val[features_3])))
OLS_3_test_R2.append(R_squared(y_test, OLS_3.predict(x_test[features_3])))

OLS_3_test_t_mse.append(mean_squared_error(y_test_t, OLS_3.predict(x_test_t[features_3])))
OLS_3_test_t_R2.append(R_squared(y_test_t, OLS_3.predict(x_test_t[features_3])))

OLS_3_test_b_mse.append(mean_squared_error(y_test_b, OLS_3.predict(x_test_b[features_3])))
OLS_3_test_b_R2.append(R_squared(y_test_b, OLS_3.predict(x_test_b[features_3])))

print(f'The total training set has MSE {mean_squared_error(y_train, OLS_3.predict(x_train[features_3]))}')
print(f'The total validation set has MSE {mean_squared_error(y_val, OLS_3.predict(x_val[features_3]))}')
print(f'The total test set has MSE {mean_squared_error(y_test, OLS_3.predict(x_test[features_3]))}')
print(f'The total training set has r^2 {R_squared(y_train, OLS_3.predict(x_train[features_3]))}')
print(f'The total validation set has r^2 {R_squared(y_val, OLS_3.predict(x_val[features_3]))}')
print(f'The total test set has r^2 {R_squared(y_test, OLS_3.predict(x_test[features_3]))}\n')

print(f'The top 1k test set has MSE {mean_squared_error(y_test_t, OLS_3.predict(x_test_t[features_3]))}')
print(f'The top 1k test set has r^2 {R_squared(y_test_t, OLS_3.predict(x_test_t[features_3]))}\n')

print(f'The bottom 1k test set has MSE {mean_squared_error(y_test_b, OLS_3.predict(x_test_b[features_3]))}')
print(f'The bottom 1k test set has r^2 {R_squared(y_test_b, OLS_3.predict(x_test_b[features_3]))}')

The total training set has MSE 0.015894677700940815
The total validation set has MSE 0.02657529108184017
The total test set has MSE 0.0362750727239179
The total training set has r^2 0.003989748312871355
The total validation set has r^2 -0.001847618702224274
The total test set has r^2 0.0020615522965783395

The top 1k test set has MSE 0.015601747395857894
The top 1k test set has r^2 -0.0020383520128555155

The bottom 1k test set has MSE 0.08297433207552728
The bottom 1k test set has r^2 0.0011847044500689075


In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
# Import the class NN from NN_implementations
from ipynb.fs.defs.NN_implementations import NN

In [26]:
model = NN()

In [31]:
# Record annual results (30, 3) = (year, model)
loss_train = np.zeros((30, 3))
loss_val = np.zeros((30, 3))
loss_test = np.zeros((30, 3))
loss_test_t = np.zeros((30, 3))
loss_test_b = np.zeros((30, 3))
R2_train = np.zeros((30, 3))
R2_val = np.zeros((30, 3))
R2_test = np.zeros((30, 3))
R2_test_t = np.zeros((30, 3))
R2_test_b = np.zeros((30, 3))

# Dictionary keys are 'year_model' (e.g. '1975_0' means training till 1975 and validation starts on 1975)
y_train_pred_dict = {}
y_val_pred_dict = {}
# Dictionary keys are 'year_model' (e.g. '1987_0' means the first model testing 1987)
y_pred_dict = {}
y_pred_t_dict = {}
y_pred_b_dict = {}

# Record monthly results (30, 12, 3) = (year, month, model)
loss_testM = np.zeros((30, 12, 3))
loss_test_tM = np.zeros((30, 12, 3))
loss_test_bM = np.zeros((30, 12, 3))
R2_testM = np.zeros((30, 12, 3))
R2_test_tM = np.zeros((30, 12, 3))
R2_test_bM = np.zeros((30, 12, 3))

# Dictionary keys are 'year_month_model' (e.g. '1987_0_3' means the fourth model testing Jan 1987)
y_predM_dict = {}
y_pred_tM_dict = {}
y_pred_bM_dict = {}

# Specify the hyperparameters
L1_val = 0
L2_val = 0.01
dropout = 0
lr_val = 1e-3
bs_val = 256

In [32]:
# Define and compile 10 models for ensemble later
model_dict = {}
for i in range(3):
    seed_val = 120 + i
    model_dict[str(i)] = model.call(
                                model_input = keras.layers.Input(shape=(920, )),
                                n_layers = 3,
                                layers_dim = [32, 16, 8],
                                activation = 'relu',
                                BatchNormalization = True,
                                L1_lambda = L1_val,
                                L2_lambda = L2_val,
                                dropout_rate = dropout,
                                seed = seed_val)
    model_dict[str(i)].compile(
        loss = keras.losses.MeanSquaredError(),
        optimizer=keras.optimizers.Adam(lr_val),
        metrics=[R_squared]
        )

In [33]:
mse = tf.keras.losses.MeanSquaredError()

In [30]:
start_val = np.datetime64('1975-01-31')
start_test = np.datetime64('1987-01-31')
end_test = np.datetime64('1987-12-31')

In [76]:
# Divide stocks into deciles according to y_predM and use y_testM to calculate annualized Sharpe ratio
def make_decile(x_testM, y_testM, y_predM):
    x_testM = pd.concat([x_testM, y_testM, y_predM], axis=1)
    x_testM['DistinctRank'] = x_testM.Pred(method='first')
    x_testM_grouped = x_testM.groupby(pd.qcut(x_testM.DistinctRank, 10, labels=False))

    # decile['0'] is lowest decile, decile['9'] highest
    decile = {}
    for key, group in x_testM_grouped:
        decile[str(key)] = pd.DataFrame(group).sort_values(by=['Pred'], ascending=False, ignore_index=True)
            
    pred = []
    std = []
    avg = []
    Sharpe = []
    for i in range(10): # 10 deciles
        pred.append(np.mean(decile[str(i)]['Pred']))
        avg.append(np.mean(decile[str(i)]['RET'])*12)
        std.append(np.std(decile[str(i)]['RET'])*sqrt(12))
        Sharpe.append(avg[-1] / std[-1])
            
    return pred, avg, std, Sharpe

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# Use start years for naming keys in prediction dictionaries
start_val_year = 1975
start_test_year = 1987
# Construct monthly start and end dates for test set
start_test_ym = '1987-01'
end_test_ym = '1988-02'
all_months = np.arange(start_test_ym, end_test_ym, dtype='datetime64[M]').astype('datetime64[D]')

for i in range(3): # i is model
    # Housekeeping
    gc.collect()
    keras.backend.clear_session()
    
    # Train the model
    history = model_dict[str(i)].fit(x_train, y_train, validation_data = (x_val, y_val),
                               # change batch_size and epoch
                               batch_size=bs_val, epochs=100
                               # optional early stop
                               ,callbacks=[earlystop]
                               )
    
    # Save weights of all models for all years
    # e.g. 'weights_1987_0' means the first model weights when the test set starts on 1987, so training ends on 1987-13=1974
    # model_dict[str(i)].save_weights(f'Results_log\\seed129_{start_test_year}_' + str(i) + '.h5')
    
    # Find model predictions for entire/top/bottom
    y_train_pred_dict[f'{start_val_year}_' + str(i)] = model_dict[str(i)].predict(x_train, batch_size=x_train.shape[0])
    y_val_pred_dict[f'{start_val_year}_' + str(i)] = model_dict[str(i)].predict(x_val, batch_size=x_val.shape[0])
    y_pred_dict[f'{start_test_year}_' + str(i)] = model_dict[str(i)].predict(x_test, batch_size=x_test.shape[0])
    y_pred_t_dict[f'{start_test_year}_' + str(i)] = model_dict[str(i)].predict(x_test_t, batch_size=x_test_t.shape[0])
    y_pred_b_dict[f'{start_test_year}_' + str(i)] = model_dict[str(i)].predict(x_test_b, batch_size=x_test_b.shape[0])
    predY, avgY, stdY, SharpeY = make_decile(x_test, y_test, pd.DataFrame(y_pred_dict[f'{start_test_year}_' + str(i)]), columns=['Pred'])
    performace = pd.DataFrame(data={'Pred': pred, 'Avg': avg, 'Std': std, 'SR': Sharpe})
    print(performance)
        
    # Record loss and R^2: train and validation
    loss_train[0, i] = mse(y_train, y_train_pred_dict[f'{start_val_year}_' + str(i)])
    R2_train[0, i] = R_squared(y_train, y_train_pred_dict[f'{start_val_year}_' + str(i)])
    loss_val[0, i] = mse(y_val, y_val_pred_dict[f'{start_val_year}_' + str(i)])
    R2_val[0, i] = R_squared(y_val, y_val_pred_dict[f'{start_val_year}_' + str(i)])
    # Record loss and R^2: test, top/bottom test
    loss_test[0, i] = mse(y_test, y_pred_dict[f'{start_test_year}_' + str(i)])
    R2_test[0, i] = R_squared(y_test, y_pred_dict[f'{start_test_year}_' + str(i)])
    loss_test_t[0, i] = mse(y_test_t, y_pred_t_dict[f'{start_test_year}_' + str(i)])
    R2_test_t[0, i] = R_squared(y_test_t, y_pred_t_dict[f'{start_test_year}_' + str(i)])
    loss_test_b[0, i] = mse(y_test_b, y_pred_b_dict[f'{start_test_year}_' + str(i)])
    R2_test_b[0, i] = R_squared(y_test_b, y_pred_b_dict[f'{start_test_year}_' + str(i)]) 
    
    decile = {}
    std = {}
    avg = {}
    Sharpe = {}
    for j in range(12): # j is month
        # Get entire, top, and bottom test sets
        start_testM = all_months[j]
        end_testM = all_months[j+1]
        x_testM, y_testM, x_test_tM, y_test_tM, x_test_bM, y_test_bM = interactions(
                                                data[(data['DATE'] >= start_testM) & (data['DATE'] <= end_testM)],
                                                data_ma[(data_ma['yyyymm'] >= start_testM) & (data_ma['yyyymm'] <= end_testM)],
                                                characteristics, ma_predictors)
        
        # Find model predictions for entire/top/bottom WITHOUT batch_size
        y_predM_dict[f'{start_test_year}_' + str(j) + '_' + str(i)] = model_dict[str(i)].predict(x_testM, batch_size=x_testM.shape[0])
        y_pred_tM_dict[f'{start_test_year}_' + str(j) + '_' + str(i)] = model_dict[str(i)].predict(x_test_tM, batch_size=x_test_tM.shape[0])
        y_pred_bM_dict[f'{start_test_year}_' + str(j) + '_' + str(i)] = model_dict[str(i)].predict(x_test_bM, batch_size=x_test_bM.shape[0])
        
        # Rename for easier reference later; RET is excess return
        y_predM = pd.DataFrame(y_predM_dict[f'{start_test_year}_' + str(j) + '_' + str(i)], columns=['Pred'])
        y_pred_tM = pd.DataFrame(y_pred_tM_dict[f'{start_test_year}_' + str(j) + '_' + str(i)], columns=['Pred'])
        y_pred_bM = pd.DataFrame(y_pred_bM_dict[f'{start_test_year}_' + str(j) + '_' + str(i)], columns=['Pred'])
        
        # Record loss and R^2
        loss_testM[0, j, i] = mse(y_testM, y_predM)
        R2_testM[0, j, i] = R_squared(y_testM, y_predM)
        loss_test_tM[0, j, i] = mse(y_test_tM, y_pred_tM)
        R2_test_tM[0, j, i] = R_squared(y_test_tM, y_pred_tM)
        loss_test_bM[0, j, i] = mse(y_test_bM, y_pred_bM)
        R2_test_bM[0, j, i] = R_squared(y_test_bM, y_pred_bM)  
    
        pred, std, avg, Sharpe = make_decile(x_testM, y_testM, y_predM)
        performace = pd.DataFrame(data={'Pred': pred, 'Avg': avg, 'Std': std, 'SR': Sharpe})
        performace.to_csv(f'Results_log\\{start_test_year}_perf.csv', mode='a')



In [ ]:
def get_SR_table()

# Recursively do OLS_3 and refit NN models

In [ ]:
def get_datetime64(date_digits):
    date_str = str(date_digits)
    date_str = date_str[0:4] + '-' + date_str[4:6] + '-' + date_str[6:8]
    return date_str

In [ ]:
# start_val = np.datetime64('1975-01-31')
# start_test = np.datetime64('1987-01-31')
# end_test = np.datetime64('1987-12-31')
start_test_year = 1987
end_test_year = 1988
start_val_numer = 19750131
start_test_numer = 19870131
end_test_numer = 19871231

In [ ]:
# Record the sizes of training, validation, test sets
train_shape = [0]*30
val_shape = [0]*30
test_shape = [0]*30
train_shape[0] = 479467
val_shape[0] = 773887
test_shape[0] = 83323

In [ ]:
# There are in total 30 OOS years to test (1987-2016)
# Train every year, but test every month and every year
for year in range(1, 30):
    
    gc.collect()
    keras.backend.clear_session()
    
    # Set the correct dates
    start_val_prev_numer = start_val_numer
    start_test_prev_numer = start_test_numer
    end_test_prev_numer = end_test_numer
    start_val_numer = start_val_numer + 10000
    start_test_numer = start_test_numer + 10000
    end_test_numer = end_test_numer + 10000
    start_val_prev = get_datetime64(start_val_prev_numer)
    start_test_prev = get_datetime64(start_test_prev_numer)
    end_test_prev = get_datetime64(end_test_prev_numer)
    start_val = get_datetime64(start_val_numer)
    start_test = get_datetime64(start_test_numer)
    end_test = get_datetime64(end_test_numer)
    print(start_val_prev, start_test_prev, end_test_prev, start_val, start_test, end_test)
    
    # Add one more year to training
    x_train_add, y_train_add, _, _, _, _ = interactions(data[(data['DATE'] < start_val) & (data['DATE'] >= start_val_prev)],
                                           data_ma[(data_ma['yyyymm'] < start_val) & (data_ma['yyyymm'] >= start_val_prev)],
                                           characteristics, ma_predictors)
    x_train = pd.concat([x_train, x_train_add], ignore_index=True)
    y_train = pd.concat([y_train, y_train_add], ignore_index=True)
    train_shape[year] = x_train.shape[0]
    
    # Since x,y_val has no more date inside, we will just get them again
    x_val, y_val, _, _, _, _ = interactions(data[(data['DATE'] < start_test) & (data['DATE'] >= start_val)],
                                                                    data_ma[(data_ma['yyyymm'] < start_test) & (data_ma['yyyymm'] >= start_val)],
                                                                    characteristics, ma_predictors)
    val_shape[year] = x_val.shape[0]

    # Change the test set to the next year
    x_test, y_test, x_test_t, y_test_t, x_test_b, y_test_b = interactions(data[(data['DATE'] >= start_test) & (data['DATE'] <= end_test)],
                                                                          data_ma[(data_ma['yyyymm'] >= start_test) & (data_ma['yyyymm'] <= end_test)],
                                                                          characteristics, ma_predictors)
    test_shape[year] = x_test.shape[0]
    
    # Do OLS_3 first
    # Replace the OLS model every iteration (could put in dictionary if want to save all 30 of them)
    OLS_3 = LinearRegression().fit(x_train[features_3], y_train)
    
    OLS_3_train_mse.append(mean_squared_error(y_train, OLS_3.predict(x_train[features_3])))
    OLS_3_val_mse.append(mean_squared_error(y_val, OLS_3.predict(x_val[features_3])))
    OLS_3_test_mse.append(mean_squared_error(y_test, OLS_3.predict(x_test[features_3])))
    OLS_3_train_R2.append(R_squared(y_train, OLS_3.predict(x_train[features_3])))
    OLS_3_val_R2.append(R_squared(y_val, OLS_3.predict(x_val[features_3])))
    OLS_3_test_R2.append(R_squared(y_test, OLS_3.predict(x_test[features_3])))
    
    #OLS_3_val_t_mse.append(mean_squared_error(y_val_t, OLS_3.predict(x_val_t[features_3])))
    OLS_3_test_t_mse.append(mean_squared_error(y_test_t, OLS_3.predict(x_test_t[features_3])))
    #OLS_3_val_b_mse.append(mean_squared_error(y_val_b, OLS_3.predict(x_val_b[features_3])))
    OLS_3_test_b_mse.append(mean_squared_error(y_test_b, OLS_3.predict(x_test_b[features_3])))
    #OLS_3_train_t_R2_demeaned.append(r2_score(y_train_t, OLS_3.predict(x_train_t[features_3])))
    #OLS_3_train_t_R2.append(R_oos(y_train_t, OLS_3.predict(x_train_t[features_3])))
    #OLS_3_val_t_R2.append(R_oos(y_val_t, OLS_3.predict(x_val_t[features_3])))
    OLS_3_test_t_R2.append(R_squared(y_test_t, OLS_3.predict(x_test_t[features_3])))
    #OLS_3_val_b_R2.append(R_oos(y_val_b, OLS_3.predict(x_val_b[features_3])))
    OLS_3_test_b_R2.append(R_squared(y_test_b, OLS_3.predict(x_test_b[features_3])))
    print(OLS_3_train_R2[-1], OLS_3_val_R2[-1],  OLS_3_val_t_R2[-1], OLS_3_val_b_R2[-1], OLS_3_test_R2[-1], OLS_3_test_t_R2[-1], 
         OLS_3_test_b_R2[-1])
    
    for i in range(10):
        # Housekeeping
        keras.backend.clear_session()
        gc.collect()
        
        # This refits the model used before
        history = model_dict[str(i)].fit(x_train, y_train, validation_data = (x_val, y_val),
                               # change batch_size and epoch
                               batch_size=bs_val, epochs=100
                               # optional early stop
                               ,callbacks=[earlystop]
                               )
    
        loss_val_t[year, i], R2_val_t[year, i] = model_dict[str(i)].evaluate(x_val_t, y_val_t, batch_size=bs_val)
        loss_val_b[year, i], R2_val_b[year, i] = model_dict[str(i)].evaluate(x_val_b, y_val_b, batch_size=bs_val)
        loss_test[year, i], R2_test[year, i] = model_dict[str(i)].evaluate(x_test, y_test, batch_size=bs_val)
        loss_test_t[year, i], R2_test_t[year, i] = model_dict[str(i)].evaluate(x_test_t, y_test_t, batch_size=bs_val)
        loss_test_b[year, i], R2_test_b[year, i] = model_dict[str(i)].evaluate(x_test_b, y_test_b, batch_size=bs_val)

    #print(loss_val_t[year, :], loss_val_b[year, :], loss_test[year, :], loss_test_t[year, :], loss_test_b[year, :])
    print(R2_val_t[year, :], R2_val_b[year, :], R2_test[year, :], R2_test_t[year, :], R2_test_b[year, :])    
        
        

In [ ]:
# Some references for model.evaluate()
# https://stackoverflow.com/questions/50723287/meaning-of-batch-size-in-model-evaluate (floating point error)
# https://stackoverflow.com/questions/49359489/how-are-metrics-computed-in-keras (val metric)

print(OLS_3_train_R2)
print(OLS_3_val_R2)
print(OLS_3_val_t_R2)
print(OLS_3_val_b_R2)
print(OLS_3_test_R2)
print(OLS_3_test_t_R2)
print(OLS_3_test_b_R2)

In [ ]:
print(R2_val_t[0:12, :])
print(R2_val_b[0:12, :])
print(R2_test[0:12, :])
print(R2_test_t[0:12, :])
print(R2_test_b[0:12, :])

In [ ]:
for i in range(10):
    model_dict[str(i)].save_weights(f'model_weights_{i}.h5')

In [ ]:
model_dict['9'].evaluate(x_val_t, y_val_t, batch_size=10000)

In [ ]:
np.mean(R2_test[0:12,:], axis=1) - np.array(OLS_3_test_R2)

In [ ]:
np.mean(R2_test_t[0:12,:], axis=1) - np.array(OLS_3_test_t_R2)

In [ ]:
np.mean(R2_test_b[0:12,:], axis=1) - np.array(OLS_3_test_b_R2)